<a href="https://colab.research.google.com/github/moridin04/Data-Science-Project/blob/main/Data_Science_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Science - Course Project**

**Analyzing Factors Influencing Writing Productivity Through Machine Learning Models**

## **PHASE 1: Data Preparation**

Importing Libraries

Dataset

Preprocessing

Cleaning Data

Transforming Data

Handling Missing Values

Converting Text to Numeric

Descriptive Statistics

Visualization Tools

## **PHASE 2: Exploratory Data Analysis (EDA)**

### Statistical Analysis

Correlation

t-test

Regression

GroupBy

Aggregation

Central Tendency (Mean, Median, Mode)

## **PHASE 3: Machine Learning**

Select Features (X) and Target (y)

Train-Test Split

Feature Scaling (fit on Train, transform on Test)

Logistic Regression Model Training

Model Evaluation (Accuracy, Confusion Matrix)

Visualization of Results (Heatmap)

# **Final Conclusion**